# IT3103 Assignment 1

### Importing the necessary libraries

In [1]:
import tensorflow as tf
import tensorflow.keras as keras
import matplotlib.pyplot as plt
import os
import numpy as np

### Importing the dataset
Imports the datasets to be used when training the model

In [2]:
# Get the file from the link
dataset_URL = 'https://nyp-aicourse.s3.ap-southeast-1.amazonaws.com/it3103/datasets/fruits.zip'

# Extract the file into a folder
path_to_zip = tf.keras.utils.get_file('fruits.zip', origin=dataset_URL, extract=True, cache_dir='.')

# Set the path to look for the files
dataset_dir = os.path.join(os.path.dirname(path_to_zip), "fruits_extracted/fruits")
dataset_dir_valid = os.path.join(os.path.dirname(path_to_zip), "fruits_extracted/fruits")

### Extracting datasets 

In [3]:
# Training Directory
trainingDirectory = os.path.join(dataset_dir, "train")

# Validation Directory
validationDirectory = os.path.join(dataset_dir_valid, "valid")

### Creating the Model

In [4]:
def make_model():
    model = keras.Sequential()
    model.add(keras.layers.Input(shape=(128,128, 3)))
    model.add(keras.layers.Rescaling(1./255))
    model.add(keras.layers.Conv2D(32, (3, 3), activation="relu"))
    model.add(keras.layers.MaxPooling2D((2,2)))
    model.add(keras.layers.Conv2D(64, (3, 3), activation="relu"))
    model.add(keras.layers.MaxPooling2D((2,2)))
    model.add(keras.layers.Conv2D(128, (3, 3), activation="relu"))
    model.add(keras.layers.MaxPooling2D((2,2)))
    model.add(keras.layers.Conv2D(128, (3, 3), activation="relu"))
    model.add(keras.layers.MaxPooling2D((2,2)))
    model.add(keras.layers.Flatten())
    model.add(keras.layers.Dense(512, activation="relu"))
    model.add(keras.layers.Dropout(0.3))
    model.add(keras.layers.Dense(256, activation="relu"))
    model.add(keras.layers.Dropout(0.3))
    model.add
    model.add(keras.layers.Dense(6, activation="softmax"))

    return model

model = make_model()
print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 rescaling (Rescaling)       (None, 128, 128, 3)       0         
                                                                 
 conv2d (Conv2D)             (None, 126, 126, 32)      896       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 63, 63, 32)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 61, 61, 64)        18496     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 30, 30, 64)       0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 28, 28, 128)       7

### Adding Optimizer and Loss Function

In [5]:
# Sparse Categorical Crossentropy is used when truth labels are integer coded and have multiple classes
# Categorical Crossentropy is used when it is one hot coded
model.compile(loss='sparse_categorical_crossentropy',
              optimizer=keras.optimizers.Adam(learning_rate=1e-4),
              metrics=['accuracy'])

### Data Preprocessing

In [6]:
imgHeight, imgWidth = 128, 128
batchSize = 32

# Resizing all images into the smae size
imageSize = (imgHeight, imgWidth)

trainingDataset = keras.preprocessing.image_dataset_from_directory(
    trainingDirectory,
    seed=1,
    image_size=imageSize,
    batch_size=batchSize,
    label_mode="int"
)

validationDataset = keras.preprocessing.image_dataset_from_directory(
    validationDirectory,
    seed=1,
    image_size=imageSize,
    batch_size=batchSize,
    label_mode="int"
)

for images, labels in trainingDataset.take(1):
    print("Images Shape: ", images.shape)
    print("Labels Shape: ", labels.shape)
    print(tf.squeeze(labels))

# print out the indices to find out the class
print(trainingDataset.class_names)
print(validationDataset.class_names)

Found 1182 files belonging to 6 classes.
Found 329 files belonging to 6 classes.
Images Shape:  (32, 128, 128, 3)
Labels Shape:  (32,)
tf.Tensor([2 3 0 3 4 3 4 4 1 2 4 0 4 1 2 1 3 5 1 3 2 5 1 3 4 3 3 0 3 0 0 0], shape=(32,), dtype=int32)
['freshapples', 'freshbanana', 'freshoranges', 'rottenapples', 'rottenbanana', 'rottenoranges']
['freshapples', 'freshbanana', 'freshoranges', 'rottenapples', 'rottenbanana', 'rottenoranges']


### Training the Model

In [7]:
def create_tb_callback():
    root_logdir = os.path.join(os.curdir, "tb_logs")

    def get_run_logdir():
        import time
        run_id = time.strftime("run_%Y_%m_%d-%H_%M_%S")
        return os.path.join(root_logdir, run_id)

    run_logdir = get_run_logdir()
    tb_callback = tf.keras.callbacks.TensorBoard(run_logdir)

    return tb_callback

model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath="bestcheckpoint.weights.h5",
    save_weights_only=True,
    monitor="val_accuracy",
    mode="max",
    save_best_only=True
)

model.fit(
    trainingDataset,
    epochs=50,
    validation_data=validationDataset,
    callbacks=[create_tb_callback(), model_checkpoint_callback]
)

Epoch 1/50
37/37 [==============================] - 4s 33ms/step - loss: 1.7579 - accuracy: 0.2530 - val_loss: 1.6675 - val_accuracy: 0.4043
Epoch 2/50
37/37 [==============================] - 1s 23ms/step - loss: 1.5155 - accuracy: 0.4205 - val_loss: 1.1933 - val_accuracy: 0.5714
Epoch 3/50
37/37 [==============================] - 1s 21ms/step - loss: 1.1403 - accuracy: 0.5525 - val_loss: 0.9912 - val_accuracy: 0.6261
Epoch 4/50
37/37 [==============================] - 1s 22ms/step - loss: 0.9705 - accuracy: 0.6261 - val_loss: 0.8340 - val_accuracy: 0.6900
Epoch 5/50
37/37 [==============================] - 1s 22ms/step - loss: 0.8618 - accuracy: 0.6633 - val_loss: 0.8422 - val_accuracy: 0.6748
Epoch 6/50
37/37 [==============================] - 1s 23ms/step - loss: 0.7888 - accuracy: 0.6920 - val_loss: 0.7120 - val_accuracy: 0.7295
Epoch 7/50
37/37 [==============================] - 1s 22ms/step - loss: 0.6930 - accuracy: 0.7403 - val_loss: 0.6856 - val_accuracy: 0.7416
Epoch 8/50
37

In [8]:
keras.models.save_model(model, filepath="cur_model.keras")

### Testing the Model

In [9]:
from enum import Enum

filename = "TestImages/apple.jpg"

test_image = keras.preprocessing.image.load_img(
    filename, target_size=(128, 128)
)

# Converts the image to numpy array
img_array = keras.preprocessing.image.img_to_array(test_image)

img_array = tf.expand_dims(img_array, 0)

# Loading the model  to do the inference
model = tf.keras.models.load_model("cur_model.keras")
predicted_label = model(img_array)

class FruitType(Enum):
    freshapples = 0
    freshbanana = 1
    freshoranges = 2
    rottenapples = 3
    rottenbanana = 4
    rottenoranges = 5

print(FruitType(np.argmax(predicted_label)).name)

freshapples


In [10]:
validationDataset.class_names

['freshapples',
 'freshbanana',
 'freshoranges',
 'rottenapples',
 'rottenbanana',
 'rottenoranges']